In [1]:
#Feature: eine Liste bestehnd aus bytes, floates oder ints
#FeatureList: eine Liste die aus Features besteht -> Nested List
#tf.train.Example: Ein Datensatz aus einem DataSet. Ist eigentlich ein Dictionary aus strings und Features -> Column, Value
#tf.train.SequenceExample: Kann auch FeatureLists enthalten, sprich nested lists

#src: https://stackoverflow.com/questions/45634450/what-are-the-advantages-of-using-tf-train-sequenceexample-over-tf-train-example

In [2]:
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras import *
import numpy as np
from functools import partial
os.chdir('..')
join = os.path.join(os.getcwd(), '_global')
sys.path.extend([join])
from _global.config import *
from _global.funcs import *

In [3]:
def _bytes_feature(value):
    # EagerTensor ist intern, daher muss er berechnet werden
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [4]:
def create_feature(imgdata_path, curr_filename, curr_bounding_boxes):
        img_path =os.path.join(imgdata_path, curr_filename)

        face_count = 0
        bbxArr = []
        for i in curr_bounding_boxes:
            bbxArr.append([*i[:4]])

        with tf.io.gfile.GFile(img_path, 'rb') as fid:
            img_bytes = fid.read()

            img_bytes, new_bbxs, img_shape = decode_img(img_bytes, bbxs=bbxArr, resize=True)
            img_bytes = tf.io.encode_jpeg(tf.cast(img_bytes, tf.uint8))
            face_count = len(new_bbxs)

        new_bbxs = [item for i in new_bbxs for item in i]

        for _ in range(MAX_BBXS * 4 - len(new_bbxs)):
            new_bbxs.append(0)

        return tf.train.Example(
            features = tf.train.Features(feature = {
                'filename': tf.train.Feature(bytes_list = tf.train.BytesList(value = [bytes(curr_filename, 'utf-8') ])),
                'shape': tf.train.Feature(int64_list = tf.train.Int64List(value = [img_shape[0], img_shape[1], img_shape[2]])),
                'image': _bytes_feature(img_bytes),
                'bbxs': tf.train.Feature(int64_list = tf.train.Int64List(value = new_bbxs)),
                'faces': tf.train.Feature(int64_list = tf.train.Int64List(value = [face_count]))
            })
        )

In [5]:
def convertToDataSet(labels_path, imgdata_path, output_path, limit = -1):
    with open(labels_path) as f:
        lines = f.readlines()
        curr_bounding_boxes = []
        curr_filename = ""
        first = True
        counter = 1
        prevWasBegin = False
        with tf.io.TFRecordWriter(output_path) as writer:
            for line in lines:
                if limit != -1 and counter >= limit: break

                line = line.replace("\n", "")
                if prevWasBegin:
                    prevWasBegin = False
                    continue
                
                if "/" in line:
                    if not first:
                        writer.write(
                            tf.train.Example.SerializeToString(
                                    create_feature(imgdata_path, curr_filename, curr_bounding_boxes)
                            )
                        )
                        curr_bounding_boxes = []
                        counter +=1
                    curr_filename = line
                    prevWasBegin = True
                    continue
                
                first = False

                line = line.strip()
                split = line.split(" ")
                next = []

                for num in split:
                    next.append(int(num))

                curr_bounding_boxes.append(next)
        print(counter)

In [6]:
#create validation dataset
convertToDataSet(VALIDATION_LABELS, VALIDATION_IMAGES, OUTPUT_VALIDATION_TFRECORD)

3226


In [6]:
#create train dataset
convertToDataSet(TRAIN_LABELS, TRAIN_IMAGES, OUTPUT_TRAIN_TFRECORD)

12880
